In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,9753
2,Huelva,Confirmados PDIA 14 días,1675
3,Huelva,Tasa PDIA 14 días,"327,95358918281795"
4,Huelva,Confirmados PDIA 7 días,631
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,5
629,Municipio de Huelva sin especificar,Total Confirmados,124
630,Municipio de Huelva sin especificar,Curados,84


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  9753.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  2738.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 803 personas en los últimos 7 días 

Un positivo PCR cada 280 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,9753.0,631.0,1675.0,510743.0,123.545501,327.953589,100.0
Huelva-Costa,5368.0,413.0,1089.0,288115.0,143.345539,377.974073,49.0
Condado-Campiña,3011.0,179.0,462.0,155057.0,115.441418,297.954946,39.0
Huelva (capital),2738.0,179.0,514.0,143663.0,124.597147,357.781753,20.0
Sierra de Huelva-Andévalo Central,1251.0,34.0,104.0,67571.0,50.317444,153.912181,12.0
Palma del Condado (La),443.0,34.0,74.0,10761.0,315.955766,687.668432,10.0
Aljaraque,367.0,33.0,70.0,21260.0,155.221072,329.256820,8.0
Lepe,852.0,67.0,187.0,27431.0,244.249207,681.710474,6.0
San Bartolomé de la Torre,36.0,10.0,21.0,3736.0,267.665953,562.098501,5.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Corteconcepción,18.0,1.0,6.0,548.0,182.481752,1094.890511,0.0
Lucena del Puerto,77.0,18.0,32.0,3371.0,533.966182,949.273213,3.0
Palma del Condado (La),443.0,34.0,74.0,10761.0,315.955766,687.668432,10.0
Lepe,852.0,67.0,187.0,27431.0,244.249207,681.710474,6.0
Niebla,112.0,14.0,28.0,4117.0,340.053437,680.106874,0.0
San Bartolomé de la Torre,36.0,10.0,21.0,3736.0,267.665953,562.098501,5.0
Ayamonte,312.0,39.0,107.0,20946.0,186.193068,510.837391,3.0
Rociana del Condado,111.0,12.0,40.0,7855.0,152.768937,509.229790,3.0
Palos de la Frontera,202.0,20.0,51.0,11289.0,177.163611,451.767207,3.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Nerva,86.0,1.0,14.0,5235.0,19.102197,267.430755,0.0,0.071429
Minas de Riotinto,64.0,1.0,13.0,3848.0,25.987526,337.837838,1.0,0.076923
Corteconcepción,18.0,1.0,6.0,548.0,182.481752,1094.890511,0.0,0.166667
Calañas,44.0,1.0,5.0,2791.0,35.829452,179.147259,0.0,0.200000
Almonte,431.0,7.0,32.0,24191.0,28.936381,132.280600,4.0,0.218750
Beas,53.0,3.0,13.0,4257.0,70.472163,305.379375,0.0,0.230769
Bollullos Par del Condado,230.0,5.0,21.0,14272.0,35.033632,147.141256,1.0,0.238095
Trigueros,126.0,3.0,11.0,7713.0,38.895371,142.616362,0.0,0.272727
Bonares,41.0,3.0,11.0,6058.0,49.521294,181.578079,1.0,0.272727
